## EMATM0047: Data Science Project
---
### Code Section S1: Image classification
---
#### Author: Siyu Liu
#### Faculty of Engineering
#### University of Bristol
</br>


#### Input:
1. The raw pictures under the Cryo-EM that are stored via the .mrc files.
2. The corresponding artificial labelled masks, stored via the .svg files.

#### Operation:
1. Import and split the dataset with 5-fold cross validation

#### Output:
1. The image classification results under the circumstance of freezing backbone

2. The image classification results under the circumstance of fine-tuned variant

Define the path of root dictionary and its auxiliary stage folders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# specify the root directory
from pathlib import Path

Root = Path('/content/drive/MyDrive/Final Project/Dataset-processed')
Stages = ['stageI', 'stageII', 'stageIII', 'stageIV'] # four stages

Import the file

In [ ]:
model_sample = []

for label, stage in enumerate(Stages): # for each stage
    for mrc in (Root / stage).glob('*.mrc'): # for all mrc file in one stage
        name = mrc.stem
        # mrc and svg have same stem
        related_svg = mrc.parent / f"{name}.svg"
        model_sample.append((str(mrc), str(related_svg), label))
print(f"{len(model_sample)} samples in total")

Split the dataset with stratified 5 folds cross validation

In [ ]:
from sklearn.model_selection import StratifiedKFold
import json

# 1 to 4
labels = [s[2] for s in model_sample]

# create the new dictionary
output_dir = (Root / '5fs CV')
output_dir.mkdir(exist_ok = True)

# use StratifiedKFold() function
cv_method = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 2025)
# split to (train, test), same distribution
cv_result = cv_method.split(model_sample, labels)

# save 5 folds to json
for fold, (train, test) in enumerate(cv_result):
    # save the file path for training
    json.dump([model_sample[i] for i in train], open(output_dir / f"fold{fold}_train.json", "w"))
    # save the file path for testing
    json.dump([model_sample[i] for i in test], open(output_dir / f"fold{fold}_test.json", "w"))

Install the needed packages in advance

In [ ]:
# need mrcfile to process .mrc and cairosvg to process .svg
try:
  import mrcfile
  print("Yes")
except:
  print("No")
  !pip install mrcfile
  !pip install cairosvg
  print("mrcfile and cairosvg installed, continue")

Use the .svg file to create the ROI patch on the original micrograph

In [ ]:
from torch.utils.data import Dataset
import json
import numpy as np
import mrcfile
import cv2
from cairosvg import svg2png
import os
import torch
from tqdm import tqdm

class ROIprocess(Dataset):
    """
    This part use the train and test json file as input,
    with the following operations:

    1. read the .mrc file, map it to [0,255] for the grayscale
    2. read the .svg file
    3. retrieve the external contours from the mask to get the ROI(region of interest)
    4. crop the original .mrc image with ROI
    5. resize the cropped .mrc image to 224x224
    6. normalize the cropped .mrc image
    7. duplicate it to rgb to satisfy the model input
    """

    def __init__(self, json_path, image_size = 224):
        """
        initialization

        argument:
        json_path: the json file saved before
        image_size: the size of the output image
        """
        self.items = json.load(open(json_path))
        self.sz = image_size
        self.mrc_cache = {}
        self.svg_cache = {}
        # preload data
        self.pre_load()

    # mrc importation function
    @staticmethod
    def mrc_import(path: str) -> np.ndarray:
        """
        import and normalize the .mrc file
        """
        with mrcfile.open(path) as mrc:
            arr = mrc.data.astype(np.float32) # float for accuracy

        # quantile normalization
        p1, p99 = np.percentile(arr, [1, 99])
        arr = np.clip(arr, p1, p99)
        arr = ((arr - p1) / max(p99 - p1, 1e-5) * 255)

        # return to uint8, standard greyscale
        return arr.astype(np.uint8)

    # the function convert svg to binary mask
    @staticmethod
    def svg2mask(svg_path: str, H: int, W: int) -> np.ndarray:
        """
        this function converts the svg to png, for convinence

        argument:
        svg_path: the path of svg file
        H: the height of the image
        W: the width of the image

        """
        # switch to png
        png = svg2png(url = svg_path, output_width = W, output_height = H)
        # must pack png as 1D numpy array
        png_convert = np.frombuffer(png, dtype = np.uint8)
        # so that we can convert it to greyscale
        mask = cv2.imdecode(png_convert, cv2.IMREAD_GRAYSCALE)
        # greyscale to binary, 0 for black and 255 for white
        return (mask > 5).astype(np.uint8) * 255

    def pre_load(self):
      """
      this function is used to pre_load the file to save time
      """
      for idx, (mrc_path, svg_path, label) in enumerate(tqdm(self.items)):
        # save mrc
        self.mrc_cache[idx] = self.mrc_import(mrc_path)
        # get image size
        image = self.mrc_cache[idx]
        # save svg
        self.svg_cache[idx] = self.svg2mask(svg_path, *image.shape)

    def __getitem__(self, idx):
        """
        process the input

        argument:
        idx: the index of the sample
        """
        # (str(mrc), str(related_svg), label)
        mrc_path, svg_path, label = self.items[idx]

        # get the mrc and svg from cache
        image = self.mrc_cache[idx]
        mask = self.svg_cache[idx]

        # find the cell mask, white dot
        # cv2.RETR_EXTERNAL outer contour only, cv2.CHAIN_APPROX_SIMPLE key points (4) on the contour only
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # no cell
        if not contours:
            # to (224,224)
            roi = cv2.resize(image, (self.sz, self.sz), interpolation = cv2.INTER_AREA)
        else:
            x_list = []
            y_list = []
            for contour in contours:
                # minimum bounding rectangle of cell mask
                x, y, w, h = cv2.boundingRect(contour)
                # four coordinates of rect
                x_list.extend([x, x + w])
                y_list.extend([y, y + h])
            # get the extreme value
            x0, x1 = min(x_list), max(x_list)
            y0, y1 = min(y_list), max(y_list)
            # get the region of interest(ROI)
            w = x1 - x0
            h = y1 - y0

            # give some padding, not to tight
            pad = int(max(w, h) * 0.05)
            # make sure still in the whole image
            H, W = image.shape
            # smallest to 0
            x0 = max(0, x0 - pad)
            y0 = max(0, y0 - pad)
            # highest to h,w
            x1 = min(W, x1 + pad)
            y1 = min(H, y1 + pad)

            # cut the ROI on mrc
            roi = image[y0: y1, x0: x1]

            # ROI must be a square
            h_roi, w_roi = roi.shape
            if h_roi != w_roi:
                # calculate the compensation
                difference = abs(h_roi - w_roi)
                pad_a = difference // 2
                pad_b = difference - pad_a
                # pad h
                if h_roi < w_roi :
                    roi = np.pad(roi, ((pad_a, pad_b), (0, 0)), mode = 'edge')
                # pad w
                else:
                    roi = np.pad(roi, ((0, 0), (pad_a, pad_b)), mode = 'edge')

            # resize the ROI to 224*224
            # ROI > 244, use cv2.INTER_AREA to keep detail
            # ROI < 224, use cv2.INTER_LINEAR to keep smooth
            interp_choice = cv2.INTER_LINEAR if min(roi.shape) < self.sz else cv2.INTER_AREA
            roi = cv2.resize(roi, (self.sz, self.sz), interpolation = interp_choice) # roi resize

        # normalization
        # [0,1]
        roi = roi.astype(np.float32) / 255

        # greyscale to rgb, [3,224,224]
        roi_rgb = np.stack([roi, roi, roi], axis = 0)

        # imagenet normalization
        mean = np.array([0.485, 0.456, 0.406])[:, None, None]
        std  = np.array([0.229, 0.224, 0.225])[:, None, None]
        roi_rgb = (roi_rgb - mean) / std

        # to tensor
        roi_rgb = torch.from_numpy(roi_rgb)

        # label to tensor, discrete number
        label_tensor = torch.tensor(label, dtype = torch.long)

        return roi_rgb, label_tensor

    def __len__(self):
        return len(self.items)

Construct the Dataloader for each fold

In [ ]:
from torch.utils.data import DataLoader
from pathlib import Path

batch_size = 16
num_workers = 2

# root and 5-fold folders
Root = Path('/content/drive/MyDrive/Final Project/Dataset-processed')
folder = (Root / '5fs CV')

def make_dataloader(fold_idx: int):
    """
    This part links to the above function to construct the dataloader.

    argument:
    fold_idx: the index of the fold
    """

    # import the fold
    train_json = folder / f"fold{fold_idx}_train.json"
    test_json = folder / f"fold{fold_idx}_test.json"

    # get tensor
    train_dataset = ROIprocess(train_json, image_size = 224)
    test_dataset = ROIprocess(test_json, image_size = 224)

    # allocate a random seed for each fold
    gen = torch.Generator().manual_seed(2025 + fold_idx)

    train_dataloader = DataLoader(train_dataset, batch_size = batch_size, generator = gen,
                    shuffle = True, num_workers = num_workers, pin_memory = True, drop_last = True)

    test_dataloader = DataLoader(test_dataset, batch_size = batch_size, generator = gen,
                    shuffle = False, num_workers = num_workers, pin_memory = True)

    print(f"Processing fold {fold_idx}, get {len(train_dataset)} training samples, {len(test_dataset)} test samples")

    return train_dataloader, test_dataloader

specift the device, cuda or cpu

In [ ]:
import torch

# specify the device, use GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
# create all dataloader here, shorten the training time
all_dataloader = []

for fold_idx in range(5):
    print(f"Working on the fold {fold_idx}")
    train_dataloader, test_dataloader = make_dataloader(fold_idx)
    all_dataloader.append((train_dataloader, test_dataloader))
    print(f"Finish the fold {fold_idx}")

The model construction and training section

In [ ]:
import torchvision.models as models
import torch.nn as nn
import numpy as np
from tqdm import tqdm

# store the best weight
fold_accuracy = []
best_weight_path = Root / 'best_weight'
best_weight_path.mkdir(exist_ok = True)

# one fold pair
for fold_idx, (train_dataloader, test_dataloader) in enumerate(all_dataloader):

    # introduce the model
    model = models.mobilenet_v2(weights = 'IMAGENET1K_V1')

    # we want 4 classes, original is 1000
    model.classifier[1] = nn.Linear(model.last_channel, 4)

    # freeze all gradient update except the last
    for name, p in model.named_parameters():
        if not name.startswith('classifier.1'):
            p.requires_grad = False

    model.to(device)

    # the lost function
    criterion = nn.CrossEntropyLoss()

    # AdamW for optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3, weight_decay = 1e-4)

    best_accuracy = 0
    epochs = 5

    for epoch in range(1, epochs + 1):
        # model training
        model.train()
        for x, y in tqdm(train_dataloader, desc = f"Fold {fold_idx}, epoch {epoch} [Train]"):
            # non-blocking + pin_memory, faster
            x = x.to(device, non_blocking = True).float()
            y = y.to(device, non_blocking = True).long()

            optimizer.zero_grad()

            logits = model(x)

            loss = criterion(logits, y)

            loss.backward()

            optimizer.step()

        # model evaluation phase
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for x, y in tqdm(test_dataloader, desc = f"Fold {fold_idx}, epoch {epoch} [Val]"):
                # non-blocking + pin_memory, faster
                x = x.to(device, non_blocking = True).float()
                y = y.to(device, non_blocking = True).long()
                # predicted label, (batch_size,)
                pred = model(x).argmax(1)
                # how many are correct
                correct += (pred == y).sum().item()
                # how many in total
                total += y.size(0)

        accuracy = correct / total
        print(f"The fold {fold_idx}, Epoch {epoch:02d} / {epochs}, validation accuracy: {accuracy:.4f}")

        # save the weight of the current best epoch accuracy
        if accuracy > best_accuracy:
          best_accuracy = accuracy
          weight_path = best_weight_path / f'fold{fold_idx}_best.pth'
          torch.save(model.state_dict(), weight_path)
          print(f"Best model updated, save to {weight_path}")

    # best epoch accuracy
    print(f"The best accuracy of fold {fold_idx} is: {best_accuracy:.4f} \n")
    fold_accuracy.append(best_accuracy)

print(f"The mean accuracy of 5 folds is {np.mean(fold_accuracy)}")

Print the conufusion matrix of the best epoch of each fold

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import torch

def confusion_print(model, test_dataloader, classes, fold_idx, save_path = None):
  """
  this function prints the confusion matrix according to weights saved before

  argument:
  model: MobileNet V2
  test_dataloader: the test dataloader
  classes: 4
  fold_idx: the index of the fold
  """
  model.eval()
  preds = []
  labels = []
  with torch.no_grad():
    for x, y in test_dataloader:
      x = x.to(device).float()
      y = y.to(device).long()
      # the predicted label
      pred = model(x).argmax(1)
      # add to list
      preds.append(pred.cpu().numpy())
      # ground truth to list
      labels.append(y.cpu().numpy())
  # concat list to one-dimensional vector
  y_pred = np.concatenate(preds)
  y_true = np.concatenate(labels)

  # generate confusion matrix
  confusion = confusion_matrix(y_true, y_pred, labels = list(range(len(classes))))
  # display it
  output = ConfusionMatrixDisplay(confusion, display_labels = classes)
  # show the confusion matrix
  fig, ax = plt.subplots(figsize = (6, 6))
  output.plot(ax = ax, cmap = 'Blues', colorbar = False)
  ax.set_title(f"Fold_{fold_idx + 1}_Confusion_Matrix")

  # save the picture
  save_path = save_path / f'fold{fold_idx + 1}_confusion matrix.png'
  plt.savefig(save_path)

  plt.show()
  plt.close()

In [ ]:
classes = ['class 1', 'class 2', 'class 3', 'class 4']
best_weight_path = Root / 'best_weight'

for fold_idx, (train_dataloader, test_dataloader) in enumerate(all_dataloader):
    # same model but no weights
    model = models.mobilenet_v2(weights = None)
    # still 4 classes
    model.classifier[1] = nn.Linear(model.last_channel, 4)
    # load the best weight
    model.load_state_dict(torch.load(best_weight_path / f'fold{fold_idx}_best.pth'))
    model.to(device)

    confusion_print(model, test_dataloader, classes, fold_idx, save_path = Root / 'best_weight')

In [ ]:
# The above model is the version which freezes all layers except the final classifier
# Now there will be a fine-tune model

import torchvision.models as models
import torch.nn as nn
import numpy as np
from tqdm.auto import tqdm
import torch.optim as optim

fold_accuracy = []
best_weight_path = Root / 'best_weight_finetune'
best_weight_path.mkdir(exist_ok = True)


# one fold pair
for fold_idx, (train_dataloader, test_dataloader) in enumerate(all_dataloader):

    # introduce the model
    model = models.mobilenet_v2(weights = 'IMAGENET1K_V1')
    # we want 4 classes
    model.classifier[1] = nn.Linear(model.last_channel, 4)
    model.to(device)

    # unfreeze all
    for p in model.parameters():
        p.requires_grad = True

    best_accuracy = 0
    epochs = 5

    # the lost function
    criterion = nn.CrossEntropyLoss()

    # optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr = 5e-5, weight_decay = 1e-4)

    # add a scheduler, the cosine annealing
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = epochs, eta_min = 0.0)

    for epoch in range(1, epochs + 1):

        # model training phase
        model.train()
        for x, y in tqdm(train_dataloader, desc = f"Fold {fold_idx}, epoch {epoch} [Train]"):
            x = x.to(device).float()
            y = y.to(device).long()
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

        # model evaluation phase
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for x, y in tqdm(test_dataloader, desc = f"Fold {fold_idx}, epoch {epoch} [Val]"):
                x = x.to(device).float()
                y = y.to(device).long()
                # get predicted label
                pred = model(x).argmax(1)
                # how many are correct
                correct += (pred == y).sum().item()
                # how many in total
                total += y.size(0)
        accuracy = correct / total
        print(f"The fold {fold_idx}, Epoch {epoch:02d} / {epochs}, validation accuracy: {accuracy:.4f}")

        # update scheduler
        scheduler.step()

        # save the weight of the current best accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            weight_path = best_weight_path / f'fold{fold_idx}_best.pth'
            torch.save(model.state_dict(), weight_path)
            print(f"Best model updated, save to {weight_path}")

    # best epoch accuracy
    print(f"The best accuracy of fold {fold_idx} is: {best_accuracy:.4f} \n")
    fold_accuracy.append(best_accuracy)

# mean accuracy
print(f"The mean accuracy of 5 folds is {np.mean(fold_accuracy):.4f} ± {np.std(fold_accuracy):.4f}")

In [ ]:
classes = ['class 1', 'class 2', 'class 3', 'class 4']
best_weight_path = Root / 'best_weight_finetune'

for fold_idx, (train_dataloader, test_dataloader) in enumerate(all_dataloader):
    # same model but no weights
    model = models.mobilenet_v2(weights = None)
    # still 4 classes
    model.classifier[1] = nn.Linear(model.last_channel, 4)
    # load the best weight
    model.load_state_dict(torch.load(best_weight_path / f'fold{fold_idx}_best.pth'))
    model.to(device)

    confusion_print(model, test_dataloader, classes, fold_idx, save_path = Root / 'best_weight_finetune')

### Explainable analysis with Grad-CAM

import the package first

In [ ]:
try:
  from pytorch_grad_cam import GradCAM
  print("Yes")
except:
  print("No")
  # install only, don't change CUDA and cuDNN
  !pip install --no-deps grad-cam == 1.5.5
  !pip install --no-deps ttach == 0.0.3
  print("grad-cam installed")
  from pytorch_grad_cam import GradCAM
  print("grad-cam imported")

Copy the same process of how the ROI is made

In [ ]:
from torch.utils.data import Dataset
import json
import numpy as np
import mrcfile
import cv2
from cairosvg import svg2png
import os
import torch
from tqdm import tqdm

def roi_process(mrc_path, svg_path, label, img_size = 224):
    """
    This function repeats the process of getting the ROI

    argument:
    mrc_path: the imput mrc path
    svg_path: the input svg path
    img_size: the image size
    device: the device, cuda or cpu
    """
    # import and normalize mrc
    with mrcfile.open(mrc_path, permissive = True) as mrc:
        mrc_data = mrc.data.astype(np.float32)

    # quantile normalization
    p1, p99 = np.percentile(mrc_data, [1, 99])
    mrc_data = np.clip(mrc_data, p1, p99)
    image = ((mrc_data - p1) / max(p99 - p1, 1e-5) * 255).astype(np.uint8)

    # binary mask
    W, H = mrc_data.shape
    png = svg2png(url = svg_path, output_width = W, output_height = H)
    # must pack png as 1D numpy array
    png_convert = np.frombuffer(png, dtype = np.uint8)
    # so that we can convert it to greyscale
    mask = cv2.imdecode(png_convert, cv2.IMREAD_GRAYSCALE)
    # greyscale to binary, 0 for black and 255 for white
    mask = (mask > 5).astype(np.uint8) * 255

    # ROI
    # find the cell mask, white dot
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # no cell
    if not contours:
        # to (224,224)
        roi = cv2.resize(image, (img_size, img_size), interpolation = cv2.INTER_AREA)
    else:
        x_list = []
        y_list = []
        for contour in contours:
            # minimum bounding rectangle of cell mask
            x, y, w, h = cv2.boundingRect(contour)
            # four coordinates of rect
            x_list.extend([x, x + w])
            y_list.extend([y, y + h])
        # get the extreme value
        x0, x1 = min(x_list), max(x_list)
        y0, y1 = min(y_list), max(y_list)
        # get the region of interest(ROI)
        w = x1 - x0
        h = y1 - y0

        # give some padding, not to tight
        pad = int(max(w, h) * 0.05)
        # make sure still in the whole image
        H, W = image.shape
        # smallest to 0
        x0 = max(0, x0 - pad)
        y0 = max(0, y0 - pad)
        # highest to h,w
        x1 = min(W, x1 + pad)
        y1 = min(H, y1 + pad)

        # cut the ROI on mrc
        roi = image[y0: y1, x0: x1]

        # ROI must be a square
        h_roi, w_roi = roi.shape
        if h_roi != w_roi:
            # calculate the compensation
            difference = abs(h_roi - w_roi)
            pad_a = difference // 2
            pad_b = difference - pad_a
            # pad h
            if h_roi < w_roi :
                roi = np.pad(roi, ((pad_a, pad_b), (0, 0)), mode = 'edge')
            # pad w
            else:
                roi = np.pad(roi, ((0, 0), (pad_a, pad_b)), mode = 'edge')

        # resize the ROI to 224*224
        # ROI > 244, use cv2.INTER_AREA to keep detail
        # ROI < 224, use cv2.INTER_LINEAR to keep smooth
        interp_choice = cv2.INTER_LINEAR if min(roi.shape) < img_size else cv2.INTER_AREA
        roi = cv2.resize(roi, (img_size, img_size), interpolation = interp_choice) # roi resize

    # normalization
    # [0,1]
    roi = roi.astype(np.float32) / 255

    # greyscale to rgb
    roi_rgb = np.stack([roi, roi, roi], axis = 0)

    # imagenet normalization
    mean = np.array([0.485, 0.456, 0.406])[:, None, None]
    std  = np.array([0.229, 0.224, 0.225])[:, None, None]
    roi_rgb = (roi_rgb - mean) / std

    # to tensor
    roi_rgb = torch.from_numpy(roi_rgb)

    # label to tensor, discrete number
    label_tensor = torch.tensor(label, dtype = torch.long)

    return roi_rgb, label_tensor

Grad-CAM configuration

In [ ]:
import torchvision.models as models
from pytorch_grad_cam import GradCAM
import torch.nn as nn
import numpy as np
from tqdm.auto import tqdm
# config the grad-CAM
model = models.mobilenet_v2(weights = None)
model.classifier[1] = nn.Linear(model.last_channel, 4)

# load the fine-tune weights, with a random fold
best_weight_path = Root / 'best_weight_finetune'
model.load_state_dict(torch.load(best_weight_path / f'fold4_best.pth'))

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# config gradcam
# use the last conv2d layer
cam = GradCAM(model = model, target_layers = [model.features[-1]])

generate and save the result

In [ ]:
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import matplotlib.pyplot as plt

# get the visualization
grad_cam_path = Root / 'grad_cam'
grad_cam_path.mkdir(exist_ok = True)

# extract the file from fold 4
train_dataloader, val_dataloader = all_dataloader[4]
fold4 = train_dataloader.dataset

for (mrc_path, svg_path, label) in tqdm(fold4.items[:30], desc = 'grad-cam process'):
    # get the ROI and label tensor
    tensor, label = roi_process(mrc_path, svg_path, label)

    # [3,224,224] to [1,3,224,224]
    tensor = tensor[None,:,:,:].float()

    # construct the target class (1,2,3,4) list
    targets = [ClassifierOutputTarget(label)]

    # get first heatmap, because batch_size = 1
    outcome = cam(input_tensor = tensor, targets = targets)[0]

    # tensor to numpy array, drop batch_size
    tensor2np = tensor[0].detach().cpu().numpy()

    # [3,224,224] to [224,224,3]
    tensor2np = tensor2np.transpose(1, 2, 0).astype(np.float32)

    # anti-normalization
    mean = np.array([0.485, 0.456, 0.406], dtype = np.float32)
    std  = np.array([0.229, 0.224, 0.225], dtype = np.float32)
    imgtensor2np = tensor2np * std[None, None, :] + mean[None, None, :]

    # save final result
    cam_img = show_cam_on_image(imgtensor2np, outcome, use_rgb = True)
    name = Path(mrc_path).stem
    save_path = grad_cam_path / f"{name}_camresult.png"
    plt.imsave(str(save_path), cam_img)